# Tidying a spreadsheet with Python and pandas

## Introduction

The concept of tidy data comes from the R world (see [_Hadley Wickham - "Tidy Data"_](https://vita.had.co.nz/papers/tidy-data.pdf)).

Tidy data is a standard way of mapping the meaning of a dataset to its structure.  A dataset is messy or tidy depending on how rows, columns and tables are matched up with observations,variables and types.  In tidy data:

1.  Each variable forms a column;
2.  Each observation forms a row;
3.  Each type of observational unit forms a table.

## Objective

I am going to show how to go from a “messy”—in the above sense—Excel spreadsheet to a “tidy” table (and CSV file) using the folowing data set, [_Hannah Ritchie and Max Roser (2018) - "Urbanization", Published online at OurWorldInData.org._](https://ourworldindata.org/urbanization). In this notebook, I am going to build a dataset of the urban/rural populations per country, each country being characterized by its continent/region and “sub-continent/region.”

## Excel and the pandas world

This step will lead you to a whole new world.

In [ ]:
import pandas as pd

`pandas` offers a method to extract data from a spreadsheet directly from a site without downloading the file itself.

In [ ]:
root_url = "https://population.un.org/wup/Download/Files/"
urban_rural_populations_url = root_url + "WUP2018-F01-Total_Urban_Rural.xls"
urban_rural_populations_raw = pd.read_excel(io=urban_rural_populations_url)

Let's take a look at the first lines to have a sense of the content.

In [ ]:
urban_rural_populations_raw.info()
display(urban_rural_populations_raw.head(n=20))
display(urban_rural_populations_raw.tail(n=20))

The first 15 lines are the headers of the Excel sheet, the 15th seems to contain the headers of the columns. We are going to move this row as the columns headers and keep only the relevant part of the table.

In [ ]:
urban_rural_populations = urban_rural_populations_raw[16:].copy()
urban_rural_populations.columns = (
    urban_rural_populations_raw.loc[15]
    .str.replace(', |\n| ', '_')
#     .str.replace(',', '')
#     .str.replace(' ', '_')
#     .str.replace('\n', '_')
    .str.lower()
    .tolist()
)
print(urban_rural_populations_raw.loc[15].tolist())
print(urban_rural_populations.columns.tolist())

We could keep the percentage of urban population here but for the sake of the demonstration, we are dropping it and will recompute it later.

In [ ]:
urban_rural_populations.drop(labels=['index', 'note', 'total', 'percentage_urban'], axis='columns', inplace=True)

The table looks a little bit cleaner–it does not have missing values, for example–but we are still missing the proper types for the columns. Indeed, the urban and rural populations have a type object while they are "people", therefore integers.

In [ ]:
display(urban_rural_populations.info())
display(urban_rural_populations.head())

In [ ]:
urban_rural_populations["country_code"] = urban_rural_populations["country_code"].astype(int)

for column in ["urban", "rural"]:
    urban_rural_populations[column] = urban_rural_populations[column].astype(int) * 1000

In [ ]:
display(urban_rural_populations.info())
display(urban_rural_populations.head(n=20))

From the structure of the table above, it seems that the country codes around 900 play a different role. After all, the column is called “region, subregion, country or area.” We can take a look at the content to help us in redesigning the table in a different way.
> We could also open the spreadsheet in a proper tool like LibreOffice, Excel or Google Sheets. But that is a bit less fun.

In [ ]:
# Here, I make use of the .pipe() operator as an introduction for chaining methods. Chaining
# is a slightly different way–closer to functional programming–to apply operators to objects.
# If you are interested, you can take a look at https://tomaugspurger.github.io/method-chaining
(
    urban_rural_populations
    .query("country_code >= 900")
    .loc[:, ['region_subregion_country_or_area', 'country_code']]
    .pipe(display)
)

In [ ]:
regions = ['AFRICA', 'ASIA', 'EUROPE', 'LATIN AMERICA AND THE CARIBBEAN', 'NORTHERN AMERICA', 'OCEANIA']

regions_and_sub_regions = (
    urban_rural_populations.query("country_code >= 900")
    .loc[:, "region_subregion_country_or_area"]
    .tolist()
)

# Notice the usage of sets here. It is a nice way to remove elements from an existing list
sub_regions = list(set(regions_and_sub_regions) - set(regions))
print(sub_regions)

The last part of the table corresponds to the regions and sub-regions as defined by the [United Nations](https://en.wikipedia.org/wiki/United_Nations_geoscheme). The country code is the norm [ISO-3166-1](https://en.wikipedia.org/wiki/ISO_3166-1_numeric). The spreadsheet is not built in a way which allows a region or a sub-region to be easily assigned to a given country, it is built like a drop-down list. We are going to change that in order to ensure a one-to-many mapping between regions/sub-regions on one side and countries on the other.


We are fully making use here of the fact that the spreadsheet is built with the following structure (region 1 → sub-region 1 → country 1 → country 2 → sub-region 2 → country 1 → country 2 → region 2 ...)

In [ ]:
urban_rural_populations["region"] = None
urban_rural_populations.loc[
    urban_rural_populations["region_subregion_country_or_area"].isin(regions),
    "region"
] = urban_rural_populations["region_subregion_country_or_area"]

urban_rural_populations['sub_region'] = None
urban_rural_populations.loc[
    urban_rural_populations["region_subregion_country_or_area"].isin(sub_regions),
    "sub_region"
] = urban_rural_populations["region_subregion_country_or_area"]

print("Before the filling of missing values")
display(urban_rural_populations.head(n=20))
print("After the filling of missing values")
urban_rural_populations.fillna(method='ffill', inplace=True)
display(urban_rural_populations.head(n=20))

Let's check the sub-regions.

In [ ]:
(
    urban_rural_populations
    .query("region_subregion_country_or_area in @sub_regions")
    .loc[:, ["region_subregion_country_or_area", "region", "sub_region"]]
    .sort_values(by="region")
)

Wait a second, that's not right. Where are the United States, Canada and so on?

In [ ]:
urban_rural_populations.loc[urban_rural_populations.region == 'NORTHERN AMERICA'].pipe(display)

In [ ]:
urban_rural_populations.loc[230:260]

As shown from the table above, Northern America does not have sub regions. When we forward fill the missing values, the algorithm uses the previous non-null record which turns out to be South America. Let's correct that!

In [ ]:
urban_rural_populations.loc[urban_rural_populations.region == 'NORTHERN AMERICA', 'sub_region'] = 'Northern America'

In [ ]:
all_subregions = (
    urban_rural_populations
    .loc[~urban_rural_populations["region_subregion_country_or_area"].isin(regions)]
    .loc[~urban_rural_populations["region_subregion_country_or_area"].str.match('.*(countries|Less|More)')]
    .groupby(by="region")
    .agg({"sub_region": "unique"})
)

for region, sub_regions in all_subregions.iterrows():
    print(f"{region}: {', '.join(sub_regions.array[0])}\n")

We can now create a table dedicated to countries themselves as they are going to be the atomic unit for the analysis.

In [ ]:
countries_urban_rural_populations = (
    urban_rural_populations
    .query("country_code < 900")
    .rename(columns=dict(region_subregion_country_or_area="country"))
    .set_index(keys=["country"])
)

countries_urban_rural_populations.head(n=10).append(countries_urban_rural_populations.tail(n=10))

We can check that we did not make any mistake during the cleaning process by comparing the total per region from our aggregated values and the ones provided by the genuine data themselves.

In [ ]:
df_r = countries_urban_rural_populations.groupby(by='region').agg(dict(urban='sum', rural='sum'))

df_rg = urban_rural_populations[
    urban_rural_populations.region_subregion_country_or_area.isin(regions)
][['urban', 'rural', 'region_subregion_country_or_area']].set_index('region_subregion_country_or_area')

df_r.join(df_rg, rsuffix='_g').pipe(display)

## Do we have tidy data?

Countries are grouped by region/continent as well as sub-regions. The following table is an extract for the sub-region South-Eastern Asia. It follows the Tidy Data principles:
- each variable forms a column;
- each observation forms a row;
- each type of observational unit forms a table.

In [ ]:
countries_urban_rural_populations.head()

Not exactly...

In [ ]:
tidy_countries_urban_rural_populations = (
    pd.melt(
        countries_urban_rural_populations.reset_index(),
        id_vars=["country", "region", "sub_region"],
        # Bring the populations into two columns by type
        # and numbers
        value_vars=["urban", "rural"],
        var_name="population_type",
        value_name="population")
    # Put all the area columns with the same convention
    .assign(
        country=lambda df: df["country"].str.title(),
        sub_region=lambda df: df["sub_region"].str.title(),
        region=lambda df: df["region"].str.title())
    .sort_values(by=["country", "population_type"])
    .reset_index(drop=True)
)

tidy_countries_urban_rural_populations.head()

## Is it easy to analyse?

Here are two examples of calculations of rural VS urban populations.

In [ ]:
(
    tidy_countries_urban_rural_populations
    .pivot_table(index="region", columns=["population_type"], values=["population"], aggfunc="sum")
    .droplevel(level=0, axis="columns")
    .assign(
        percentage_rural=lambda df: 100. * df["rural"] / (df["rural"] + df["urban"]))
    .sort_values(by="percentage_rural")
    # Format nicely the table
    .assign(
        urban=lambda df: df["urban"].map("{:,}".format),
        rural=lambda df: df["rural"].map("{:,}".format),
        percentage_rural=lambda df: df["percentage_rural"].map("{:3.1f}%".format))
    .rename_axis(mapper="", axis="rows")
    .rename_axis(mapper="", axis="columns")
    .rename(columns={"rural": "Rural population", "urban": "Urban population", "percentage_rural": "Percentage of<br>rural population"})
    .style.set_table_styles(
        [
            {"selector": "th.row_heading", "props": [("text-align", "left")]},
            {"selector": "th.col_heading", "props": [("text-align", "left")]},
        ]
    )
)

In [ ]:
(
    tidy_countries_urban_rural_populations
    .query("sub_region == 'South-Eastern Asia'")
    .pivot_table(index="country", columns=["population_type"], values=["population"])
    .droplevel(level=0, axis="columns")
    .assign(
        percentage_rural=lambda df: 100. * df["rural"] / (df["rural"] + df["urban"]))
    .sort_values(by="percentage_rural")
    # Format nicely the table
    .assign(
        urban=lambda df: df["urban"].map("{:,}".format),
        rural=lambda df: df["rural"].map("{:,}".format),
        percentage_rural=lambda df: df["percentage_rural"].map("{:3.1f}%".format))
    .rename_axis(mapper="", axis="rows")
    .rename_axis(mapper="", axis="columns")
    .rename(columns={"rural": "Rural population", "urban": "Urban population", "percentage_rural": "Percentage of<br>rural poulation"})
    .style.set_table_styles(
        [
            {"selector": "th.row_heading", "props": [("text-align", "left")]},
            {"selector": "th.col_heading", "props": [("text-align", "left")]},
        ]
    )
)

## Tidy data, cardinality and memory usage 

As you can see below, because of the particular structure of the tidy format—repeating the same elements many times—, the memory storage is definitely not optimal. 

In [ ]:
display(
    tidy_countries_urban_rural_populations
    .apply("nunique")
    .rename("Number of unique values")
    .to_frame()
)

display(
    tidy_countries_urban_rural_populations
    .memory_usage(deep=True)
    .rename("Memory usage in Bytes")
    .to_frame()
)

We can nevertheless do much better in terms of memory management by leveraging `pandas` categorical type.

In [ ]:
(
    tidy_countries_urban_rural_populations
    .memory_usage(deep=True)
    .rename("Memory usage in Bytes")
    .to_frame()
    .join(
    tidy_countries_urban_rural_populations
        .assign(
            region=lambda df: df["region"].astype("category"),
            sub_region=lambda df: df["sub_region"].astype("category"),
            population_type=lambda df: df["population_type"].astype("category"))
        .memory_usage(deep=True)
        .rename("Memory usage in Bytes with categorical data")
        .to_frame()
    )
)

In [ ]:
population_type_mapper = { population_type: n for n, population_type in enumerate(tidy_countries_urban_rural_populations["population_type"].unique())}

In [ ]:
(
    tidy_countries_urban_rural_populations
    .assign(population_type=lambda df: df["population_type"].map(population_type_mapper).astype("int8"))
    .head()
#     .memory_usage(deep=True)
#     .rename("Memory usage in Bytes with integer mapping")
#     .to_frame()
)

## A note on reproducibility

If you clean the data manually from the original Excel sheet, your friend, colleague or the community would never know the steps taken. Here you can literally read and reproduce the whole process as long as the original spreadsheet is the same.

This argument was also developed by Chris Moffit in the podcast [Talk Python To me](https://talkpython.fm/episodes/show/200/escaping-excel-hell-with-python-and-pandas).

## References

[Tidy Data by Hadley Wickham](https://vita.had.co.nz/papers/tidy-data.pdf)

[Modern Pandas (Part 5): Tidy Data](https://tomaugspurger.github.io/modern-5-tidy.html)

[Tidy Data in Python course on Datacamp](https://github.com/datacamp/courses-tidy-data-in-python/blob/master/chapter1.md)

[Tidy Data in Python by Jean-Nicolas Hould](http://www.jeannicholashould.com/tidy-data-in-python.html)

[pyjanitor package by Eric Ma](https://pyjanitor.readthedocs.io/)

and more...

## Annex: another example

In [ ]:
urban_populations_by_year_URL = root_url + 'WUP2018-F19-Urban_Population_Annual.xls'

In [ ]:
urban_populations_by_year_raw = pd.read_excel(io=urban_populations_by_year_URL)

In [ ]:
urban_populations_by_year_raw.head(n=20)

In [ ]:
def clean_string(dirty_string):
    return (
        str(dirty_string)
        .replace(",", "")
        .replace(" ", "_")  # It is also possible to use a regular expression here .replace('\n| ', '_')
        .replace("\n", "_")
        .lower()
    )

In [ ]:
urban_populations_by_year = urban_populations_by_year_raw.loc[16:]
urban_populations_by_year.columns = [clean_string(bare_column) for bare_column in urban_populations_by_year_raw.loc[15, :].tolist()]

years = [column for column in urban_populations_by_year.columns if column.replace(".", "").isdigit()]

tidy_urban_populations_by_year = (
    pd.melt(
        urban_populations_by_year,
        id_vars=["region_subregion_country_or_area"],
        value_vars=years,
        var_name="year",
        value_name="population")
    .assign(
        year=lambda df: df["year"].astype(float).astype(int),
        population=lambda df: df["population"].astype(float).astype(int) * 1000)
    
)

In [ ]:
countries = tidy_countries_urban_rural_populations["country"].unique()

tidy_countries_urban_rural_populations_urban_population_by_year = (
    tidy_urban_populations_by_year
    .query("region_subregion_country_or_area in @countries")
    .rename(mapper={"region_subregion_country_or_area": "country"}, axis="columns")
    .reset_index(drop=True)
)

In [ ]:
tidy_countries_urban_rural_populations_urban_population_by_year.head()

In [ ]:
tidy_countries_urban_rural_populations_urban_population_by_year.query("country == 'Malaysia'")